<a href="https://colab.research.google.com/github/jjrojas95/tesis/blob/master/Codigo_transitorios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

class Network:
  """
  Esta clase sera de utilidad para construir nuestras redes para poder hacer análisis 
  posteriores

  ENTRADAS:
  name : Es el nombre de la red

  df_lines : es el dataframe correspondiente a las lineas con las siguientes columnas:
  name,first_node,to_node,length,characteristic_impedance,propagation
  Nombre de la red, un extremo de nodo, el otro nodo extremo, longitud de la línea en metros,  Impeancia característica en Ohms, propagación en km/s
  
  df_lump_params : es el dataframe correspondiente a los parámetros concentrados 
  con las siguientes columnas:
  name,first_node,to_node,lump_type,value,init_conditions
  Nombre del parametro concentrado, un extremo del nodo, el otro nodo extremo, 
  Tipo de Parámetros concentrado (Capacitancia, Resistencia, Inductor), Valor del parametro en (microFarads, miliHenry o Ohms),
  Condiciones iniciales(Volts para capacitor o Ampere para Inductor)

  df_gens : es el dataframe correspondiente a los generadores con las siguientes columnas:
  name,first_node,to_node,imp_subsync,func
  Nombre del generador o fuente, Un extremo del nodo, el otro nodo extremo, Impedacia transitoria, función de generador(una customizada, un seno, escalón)
  """

  def __init__(self, name='', df_lines=None, df_lump_params = None, df_gens = None):
    self.name = name
    self.__col_name_lines = ['name', 'first_node', 'to_node','length', 'characteristic_impedance','propagation']
    self.__col_lines_types = ['object', 'int64', 'int64','float64', 'float64','float64']
    self.__col_name_lump_params = ['name', 'first_node', 'to_node','lump_type', 'value','init_conditions']
    self.__col_lump_params_types = ['object', 'int64', 'int64', 'object', 'float64','float64']
    self.__col_name_gens = ['name', 'first_node', 'to_node', 'imp_Trasient', 'func', 'properties', 'type']
    self.__col_gens_types = ['object', 'int64', 'int64', 'float64', 'object', 'object', 'uint8']
    self.__lines_NaN_name = 0
    self.__gens_NaN_name = 0

    
    if type(df_lines) == pd.DataFrame:
      self.df_lines = df_lines
    else:
      self.df_lines = pd.DataFrame(columns=self.__col_name_lines)
    if type(df_lump_params) == pd.DataFrame:
      self.df_lump_params = df_lump_params
    else:
      self.df_lump_params = pd.DataFrame(columns=self.__col_name_lump_params)
    if type(df_gens) == pd.DataFrame:
      self.df_gens = df_gens
    else:
      self.df_gens = pd.DataFrame(columns=self.__col_name_gens)
    
    for col_name, col_type in zip(self.__col_name_lines, self.__col_lines_types):
      self.df_lines[col_name] = self.df_lines[col_name].astype(col_type)
    for col_name, col_type in zip(self.__col_name_lump_params, self.__col_lump_params_types):
      self.df_lump_params[col_name] = self.df_lump_params[col_name].astype(col_type)
    for col_name, col_type in zip(self.__col_name_gens, self.__col_gens_types):
      self.df_gens[col_name] = self.df_gens[col_name].astype(col_type)

  def create_line(self, first_node, second_node, name = '', length = 60, c_imp = 250, prog = 300 ):
    """
    Esta función crea el elemento línea:

    ENTRADAS:
    first_node : nodo con referencia de mayor tensión

    second_node : nodo con referencia de menor tensión

    name : Nombre de la línea

    length : Longitud de la línea en metros [m]

    c_imp: impedancia característica en Ohms [Ohm]
    
    prog: Velocidad de propagación de la línea [km / s]

    Salidas:
    df_row : daframe de de la contrucción de la línea
    """
    if  name == '':
      self.__lines_NaN_name += 1
      name = 'línea ' + str(self.__lines_NaN_name)

    df_row = pd.DataFrame(data = np.array([[name, first_node, second_node, length, c_imp, prog]]), 
                          columns= self.__col_name_lines)
    for col_name, col_type in zip(self.__col_name_lines, self.__col_lines_types):
      df_row[col_name] = df_row[col_name].astype(col_type)
    
    self.df_lines = self.df_lines.append(df_row, ignore_index=True)
    
    return df_row

  def create_gen(self, first_node, second_node, name = '', func = 0, dict_func = {}, imp_Trasient = 0 , type_gen = 0):
    """
    Esta función crea el elemento línea:

    ENTRADAS:
    first_node : nodo con referencia de mayor tensión

    second_node : nodo con referencia de menor tensión

    name : Nombre del generador

    func : Función de la fuente, puede ser customizada sea con lambda o creada o puede elegir 
    entre los siguientes tipos: 
    0 : amp * sin(2 * pi * f * t) ----> donde f es la frecuencia que se debe modificar o dejar n 60 Hz y t el tiempo de simulación
    1 : amp * cuad(f * t) ---> es una onda cuadrada con frecuencia definida por el usuario en dict_func.f [Hz] y amplitud también dict_func.amp [Volts] 
    2 : amp * escal(t - t_init) ---> es una onda escalón definida su activación por el usuario en dict_func.t_init [s] y amplitud también dict_func.amp [Volts] 

    dict_func: Diccionario con los valores customisados de las funciones propias como cuadrada escalón y senoidal
    debe ser de la siguiente forma {'amp': <<valor de amplitud [V] o [A]>>, 'f': <<valor frecuencia [Hz]>>} o
    {'amp': <<valor de amplitud [V] o [A]>>, 't_init': <<tiempo inicio de escalón [s]>>}
    
    imp_Trasient: Inductancia transiroria o subtrnsitoria de la fuente [mH]

    type_gen: Tipo de la fuente 0 : Fuente de voltaje, 1 : Fuente de corriente
    
    Salidas:
    df_row : daframe de de la contrucción del generador
    """
    
    dict_valores = {0: ['amp', 'f'], 1: ['amp', 'f'], 2: ['amp', 't_init']}
    dict_explicación = {0: 'Onda senoidal', 1: 'Onda cuadrada', 2:'Onda escalón'}
    amp = 100
    f = 60
    t_init = 0
    if type(func) == int or type(func) == float:
      if int(func) in [0,1,2]:
        if (dict_valores[func][0] in dict_func.keys()) and (dict_valores[func][-1] in dict_func.keys()):
           dict_alt = {key: d[key] for key in dict_func.keys() & dict_valores}
        elif dict_valores[func][0] in dict_func.keys():
           dict_alt = {
               dict_valores[func][0]: dict_func[dict_valores[func][0]],
               'f' : f }
        elif dict_valores[func][-1] in dict_func.keys():
          dict_alt = {dict_valores[func][0]: dict_func[dict_valores[func][0]],
                      'f' : f } if dict_valores[func][-1] == 'f' else {
                          dict_valores[func][0]: dict_func[dict_valores[func][0]],
                            't_init' : t_init }
        else:
          dict_alt = {'amp': amp, 't_init': t_init} if func == 2 else {'amp': amp, 'f': f} 
      else:
        raise IndexError('El valor ' + str(func) + ' no está permitido, Los valores permitidos son ' + str(dict_explicación)) 
    elif callable(func):
      dict_alt = np.nan
    else:
      raise Exception('La variable "func" no es ninguna de las permitidas')

    if type(type_gen) == int:
      if type_gen in [0, 1]:
        pass
      else:
        raise IndexError('Los valores permitidos son 0 para fuente de Tensión y 1 Para fuente de Corriente')
    else:
      raise TypeError('La variable "type_gen" solo permite 0 o 1')
    
    if  name == '':
      self.__gens_NaN_name += 1
      name = 'generador ' + str(self.__gens_NaN_name)

    df_row = pd.DataFrame(data = np.array([[name, first_node, second_node, imp_Trasient, func, dict_alt, type_gen]]), 
                          columns= self.__col_name_gens)
    for col_name, col_type in zip(self.__col_name_gens, self.__col_gens_types):
      df_row[col_name] = df_row[col_name].astype(col_type)
    
    self.df_gens = self.df_gens.append(df_row, ignore_index=True)
    if callable(func):
      return df_row
    
    return pd.concat([df_row.drop(['properties'], axis=1), df_row['properties'].apply(pd.Series)], axis=1)


In [0]:
net = Network(name = 'prueba1')

In [3]:
net.create_gen(1,0, func = 0)

,name,first_node,to_node,imp_Trasient,func,type,amp,f
0,generador 1,1,0,0.0,0,0,100,60


In [4]:
net.df_gens.head()

,name,first_node,to_node,imp_Trasient,func,properties,type
0,generador 1,1,0,0.0,0,"{'amp': 100, 'f': 60}",0
